In [19]:
%matplotlib                    # Pyplot 그래프 새창에서 열기 옵션

UsageError: unrecognized arguments: Pyplot 그래프 새창에서 열기 옵션


In [ ]:
%matplotlib inline             # Pyplot 그래프 새창에서 열기 '취소' 옵션

In [3]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [2]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import time

t-SNE 라이브러리

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold      import TSNE

.

.

.



## 데이터 불러오기

특징값 데이터 불러와서 정상/고장 분리

In [9]:
FeatureData = pd.read_csv('./ProcessedData/FeatureData', header=None)
FeatureData.shape

(50, 360)

P value 순위 데이터 불러오기

In [33]:
P_value_Rank = pd.read_csv('./ProcessedData/P_value_Rank_lower' , header=None)
P_value_Rank

,0,1
0,35.0,8.968269e-01
1,29.0,5.858602e-01
2,42.0,4.537752e-01
3,28.0,2.915583e-01
4,25.0,1.501369e-01
5,19.0,1.380529e-01
6,18.0,1.340581e-01
7,38.0,1.122680e-01
8,49.0,1.064726e-01
9,48.0,9.660333e-02


## P value 기준 차원축소 대상 데이터(Feature data) 선정

In [34]:
# StartRank 부터 Number 만큼의 Feature
StartRank = 1
Number    = 5

SelectedFeatues = np.zeros((Number,FeatureData.shape[1]))

s = 0

for i in range(StartRank, StartRank+Number):
    
    index                = int(P_value_Rank.iloc[i-1,0])
    SelectedFeatues[s,:] = FeatureData.iloc[index,:].values
    s += 1

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(SelectedFeatues)
FeatureSelected.shape

(5, 360)

In [35]:
FeatureSelected = np.transpose(FeatureSelected)
FeatureSelected.shape

(360, 5)

.

.

.



# 차원축소(t-SNE) 기반 데이터 시각화

In [36]:
time_start   = time.time()
tsne         = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=400, random_state=1)
tsne_results = tsne.fit_transform(FeatureSelected)

print( '\n\n t-SNE done! Time elapsed: {} seconds'.format(time.time() - time_start ))


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 360 samples in 0.001s...
[t-SNE] Computed neighbors for 360 samples in 0.007s...
[t-SNE] Computed conditional probabilities for sample 360 / 360
[t-SNE] Mean sigma: 36.951505
[t-SNE] KL divergence after 250 iterations with early exaggeration: 50.910789
[t-SNE] KL divergence after 400 iterations: 0.108528


 t-SNE done! Time elapsed: 0.7779474258422852 seconds


In [37]:
pd.DataFrame(tsne_results)

,0,1
0,11.072635,8.763872
1,-2.819483,-11.294009
2,-2.036653,-11.863297
3,-15.224670,8.299601
4,-5.956230,-13.625385
...,...,...
355,8.072874,12.655957
356,9.996949,10.752888
357,8.811927,12.094079
358,-10.598871,-0.264048


In [38]:
NoOfData = int(FeatureSelected.shape[0]/2)

plt.figure(figsize=(10,5))

plt.scatter(tsne_results[:NoOfData,0], tsne_results[:NoOfData,1], marker='o', label='Normal')
plt.scatter(tsne_results[NoOfData:,0], tsne_results[NoOfData:,1], marker='o', label='Abnormal', c='r')
plt.title('t-SNE Result', fontsize=15)
plt.grid(alpha=0.5)
plt.legend(fontsize=12)
plt.xlabel('t-SNE_1')
plt.ylabel('t-SNE_2')

plt.show()